# 固有値問題


## 固有値問題とは
固有値問題をは与えられた行列の固有値および固有ベクトルを求めること。つまり、

$$
Ax=\lambda x , A\in \mathbb{C}^{n\times n}
$$

を満たす$\lambda\in \mathbb{C}, x\in \mathbb{C}^{n\times n} (x\neq 0)$を求める問題を指す。

Juliaで固有値と固有ベクトルを求める際には、LinearAlgebraに含まれている関数eigen()・eigvals()・eigvecs()で求めることができる。

|関数| 役割 |
|:-----------|:----------|
| eigen()    | 固有値・固有ベクトル両方  |
| eigvals()    | 固有値  |
| eigvecs()  | 固有ベクトル |


In [1]:
using LinearAlgebra
val, vec  = eigen([1 2; 2 1])

Eigen{Float64,Float64,Array{Float64,2},Array{Float64,1}}
values:
2-element Array{Float64,1}:
 -1.0
  3.0
vectors:
2×2 Array{Float64,2}:
 -0.707107  0.707107
  0.707107  0.707107

In [2]:
val#固有値

2-element Array{Float64,1}:
 -1.0
  3.0

In [3]:
vec#固有ベクトル

2×2 Array{Float64,2}:
 -0.707107  0.707107
  0.707107  0.707107

In [4]:
eigvals([1 2; 2 1])#固有値

2-element Array{Float64,1}:
 -1.0
  3.0

In [5]:
eigvecs([1 2; 2 1])#固有ベクトル

2×2 Array{Float64,2}:
 -0.707107  0.707107
  0.707107  0.707107

## 密行列に対する精度保証

密行列とは、行列内のほとんどの要素が0でない行列のことを指す。

ここでは、$A\in\mathbb{C}^{n\times n}(i=1,2,\cdots,n)$の全ての固有値に対する精度保証アルゴリズムを考える。
$A\in\mathbb{C}^{n\times n}(i=1,2,\cdots,n)$が対角化可能であるとするとき、$\lambda_i$に対する固有ベクトル$x^{(i)}\in\mathbb{C}^{n\times n}(i=1,2,\cdots,n)$を並べた行列

$$
X :=[x^{(1)},x^{(2)},\cdots,x^{(n)}]\in\mathbb{C}^{n\times n}
$$

によって、Aは

$$
X^{-1}AX = D = \rm{diag}(\lambda_1,\lambda_2,\cdots,\lambda_n)
$$

と対角化される。

In [6]:
using LinearAlgebra

n =1000;
A = randn(n,n);

xn,yn = eigen(A)

Eigen{Complex{Float64},Complex{Float64},Array{Complex{Float64},2},Array{Complex{Float64},1}}
values:
1000-element Array{Complex{Float64},1}:
 -31.856032129843104 + 0.0im
  -31.43274615163547 - 4.236076942834396im
  -31.43274615163547 + 4.236076942834396im
  -31.22879761997543 - 2.338483364708177im
  -31.22879761997543 + 2.338483364708177im
 -31.072508017468163 - 7.432943738656657im
 -31.072508017468163 + 7.432943738656657im
  -30.66641391742845 - 6.036697277355848im
  -30.66641391742845 + 6.036697277355848im
 -29.981674857356193 + 0.0im
  -29.92647406335874 - 9.200271082756919im
  -29.92647406335874 + 9.200271082756919im
 -29.543658146858142 - 3.6051989442853265im
                     ⋮
   29.24380620463043 + 2.421956872909018im
  29.436942525303788 - 4.585350648022406im
  29.436942525303788 + 4.585350648022406im
  29.921706967526518 - 9.77345675336165im
  29.921706967526518 + 9.77345675336165im
  30.024282842362098 - 3.452808481669574im
  30.024282842362098 + 3.452808481669574im
  30.

In [7]:
xn#固有値

1000-element Array{Complex{Float64},1}:
 -31.856032129843104 + 0.0im
  -31.43274615163547 - 4.236076942834396im
  -31.43274615163547 + 4.236076942834396im
  -31.22879761997543 - 2.338483364708177im
  -31.22879761997543 + 2.338483364708177im
 -31.072508017468163 - 7.432943738656657im
 -31.072508017468163 + 7.432943738656657im
  -30.66641391742845 - 6.036697277355848im
  -30.66641391742845 + 6.036697277355848im
 -29.981674857356193 + 0.0im
  -29.92647406335874 - 9.200271082756919im
  -29.92647406335874 + 9.200271082756919im
 -29.543658146858142 - 3.6051989442853265im
                     ⋮
   29.24380620463043 + 2.421956872909018im
  29.436942525303788 - 4.585350648022406im
  29.436942525303788 + 4.585350648022406im
  29.921706967526518 - 9.77345675336165im
  29.921706967526518 + 9.77345675336165im
  30.024282842362098 - 3.452808481669574im
  30.024282842362098 + 3.452808481669574im
  30.134126510182497 - 7.641707856754901im
  30.134126510182497 + 7.641707856754901im
   31.26981625259676

In [8]:
yn#固有ベクトル

1000×1000 Array{Complex{Float64},2}:
  -0.0395308+0.0im    0.00148113+0.0269323im    …   -0.0199246+0.0im
  -0.0165969+0.0im    -0.0508611+0.000840002im       0.0105945+0.0im
  -0.0595394+0.0im    0.00353909-0.0181443im         -0.032661+0.0im
 -0.00439333+0.0im      0.040413+0.0273283im         0.0189154+0.0im
   0.0249153+0.0im     0.0130105+0.000325555im       0.0151062+0.0im
 -0.00298528+0.0im   -0.00897008-0.00119271im   …    0.0575931+0.0im
  -0.0647066+0.0im   -0.00171651+0.0126077im        -0.0691384+0.0im
  -0.0396236+0.0im     0.0399975+0.00570224im        0.0295172+0.0im
  -0.0466728+0.0im     0.0461078+0.0551624im         0.0219306+0.0im
 0.000868785+0.0im    0.00364345+0.0342817im       0.000264576+0.0im
   0.0341458+0.0im     0.0292583+0.014819im     …   0.00950327+0.0im
  -0.0401058+0.0im     0.0182616-0.0336165im        0.00408241+0.0im
  0.00345103+0.0im    -0.0226524-0.0642984im        0.00798419+0.0im
            ⋮                                   ⋱  
  -0.0105997+0

しかし、実際のところ上記の数値計算で得られる固有ベクトル$X$は近似の値$\hat{X}\in \mathbb{C}^{n\times n}$(以下、近似固有ベクトル)である。しかし、近似固有ベクトルを対角行列に変換したものについては、対優位性が期待できる。この値を$G$とすると、

$$
G := \hat{X}^{-1}A\hat{X}
$$

となり、この$G$の対角成分が行列Aの固有値の近似となる。

In [19]:
inv(yn)*A*yn#G

1000×1000 Array{Complex{Float64},2}:
      -31.856+8.21678e-27im  …  -9.83658e-14-5.21771e-13im
  9.82547e-15+2.43416e-14im      -3.8114e-13+2.86368e-14im
  6.78624e-15-3.1912e-14im       3.94129e-13+1.36828e-13im
  5.10703e-15+9.71445e-15im      1.69198e-13-3.7792e-13im
  1.77636e-15-8.79852e-15im      2.69118e-13-2.06724e-13im
 -6.88338e-15+2.4869e-14im   …   2.40252e-13+2.08278e-13im
 -9.21485e-15-2.76446e-14im     -1.89626e-13+1.13687e-13im
  8.65974e-15+9.99201e-15im     -2.26819e-13+1.07525e-13im
  5.77316e-15-1.18794e-14im       6.4615e-14-1.81549e-13im
 -1.42109e-14-3.83702e-27im      6.39488e-14+2.82602e-14im
 -1.62093e-14-2.66454e-15im  …   1.03806e-13-4.87388e-14im
 -1.95399e-14+2.9976e-15im       4.88498e-14-7.58837e-14im
  4.21885e-15+4.27436e-15im      3.32623e-13+4.28546e-14im
             ⋮               ⋱  
 -1.80966e-14+6.66134e-16im     -2.79332e-13+8.32667e-15im
  8.90954e-15-1.26843e-14im      1.65201e-13-2.8838e-13im
  9.07607e-15+2.00395e-14im  …   2.53131e-13-8.

まず、行列積$A\cdot\hat{X}$の包含$C\in \mathbb{IC}^{n\times n}$を区間演算で求める。

In [15]:
using IntervalArithmetic

A_int = map(Interval, A);
yn_int = map(Interval, yn);

A_int*yn_int

TypeError: TypeError: non-boolean (Interval{Float64}) used in boolean context

上記のようなエラーが出てしまった(ynが原因…？)。

In [17]:
A

1000×1000 Array{Float64,2}:
  0.781418  -0.155116   -0.201596     0.563838   …  -0.467339   -1.2737
 -1.51074   -0.232573   -0.975547     1.5367        -0.554301   -1.82375
  0.910181  -0.145335    0.274611    -0.199351      -0.0836057   0.658482
 -0.372508  -0.0987555   0.755922     0.594012       0.453145    0.0285495
 -0.166972   1.56717    -0.760299     0.525878      -1.45913    -1.02668
  0.115155   0.999796    0.681397     2.36573    …  -0.835195   -0.352299
  0.284297  -0.26514     0.286718     0.768384       0.177195    0.213822
 -0.667977   0.590721    0.636893     0.0804752      0.341209   -0.475682
  0.915618  -0.717551    0.00499764  -0.359244       0.344332    0.65126
  0.460881   0.0158616   0.771098     1.22654        1.21874    -1.87826
 -0.302059   0.966392   -0.0584874   -0.726643   …  -0.220005    1.30405
 -1.82884   -1.92107    -0.864858    -2.06324       -1.46478    -0.562813
 -1.02315    1.86085    -0.486358    -0.494639       0.113376   -1.3497
  ⋮               

In [18]:
yn

1000×1000 Array{Complex{Float64},2}:
  -0.0395308+0.0im    0.00148113+0.0269323im    …   -0.0199246+0.0im
  -0.0165969+0.0im    -0.0508611+0.000840002im       0.0105945+0.0im
  -0.0595394+0.0im    0.00353909-0.0181443im         -0.032661+0.0im
 -0.00439333+0.0im      0.040413+0.0273283im         0.0189154+0.0im
   0.0249153+0.0im     0.0130105+0.000325555im       0.0151062+0.0im
 -0.00298528+0.0im   -0.00897008-0.00119271im   …    0.0575931+0.0im
  -0.0647066+0.0im   -0.00171651+0.0126077im        -0.0691384+0.0im
  -0.0396236+0.0im     0.0399975+0.00570224im        0.0295172+0.0im
  -0.0466728+0.0im     0.0461078+0.0551624im         0.0219306+0.0im
 0.000868785+0.0im    0.00364345+0.0342817im       0.000264576+0.0im
   0.0341458+0.0im     0.0292583+0.014819im     …   0.00950327+0.0im
  -0.0401058+0.0im     0.0182616-0.0336165im        0.00408241+0.0im
  0.00345103+0.0im    -0.0226524-0.0642984im        0.00798419+0.0im
            ⋮                                   ⋱  
  -0.0105997+0

区間連立1次方程式$\hat XG=C$の解集合の包含$G\supset\hat X^{-1}\cdot C$を求める。

$G$に対してゲルシュゴリンの定理を適用することを考える。

## ゲルシュゴリンの包含定理
行列$A=(A_{ij})\in \mathbb{C}^{n\times n}$について、$A$の全ての固有値$\lambda_i$は、$\bigcup_{1\le i \le n}  U_i$の内部に存在する。

$$
A = \bigcup_{1\le i \le n}  U_i
$$

ただし、

$$
U_i = \{z\in\mathbb{C}:|z-a_{ii}|\le \sum_{j\neq i}|a_{ij}|\}
$$

である。

円盤領域$U_i$をゲルシュゴリン円板といい、Aが強い優対角性を持つとき、Aの対角成分がAの固有値の良い近似となる。

今回の場合、$G=(g_{ij})$の対角成分$[g_{ii}]$を$c_i$、半径を$\rm{rad(g_{ii})}$とすると、

$$
\begin{array}{ll}
\mbox{ゲルシュゴリン円の中心:}&c_i\\
\mbox{ゲルシュゴリン円の半径:}&r_i=\sum_{j\neq i}\rm{mag}(g_{ij})+\rm{rad}(g_{ii})\\
\end{array}
$$

となる。